In [1]:
from __future__ import print_function # so print doesn't show brackets
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
import pickle
pickle.HIGHEST_PROTOCOL = 2
sys.path.append(os.path.join("..", "Libraries","QML_lib"))
from RedisSettings import *
import Evo as evo
import DataBase 
from QMD import QMD #  class moved to QMD in Library
import QML
import ModelGeneration 
import BayesF
import matplotlib.pyplot as plt
%matplotlib inline
#from pympler import asizeof
import matplotlib.pyplot as plt
paulis = ['x', 'y', 'z'] # will be chosen at random. or uncomment below and comment within loop to hard-set

import time as time 
import argparse

parser = argparse.ArgumentParser(description='Pass variables for (I)QLE.')

import warnings
warnings.filterwarnings("ignore")

In [2]:
num_runs=1
num_particles=40
num_experiments = 10
num_times_bayes = 'all'
qle=True

#initial_op_list = ['x', 'y', 'z', 'xTx', 'yTz', 'xTz']
#initial_op_list = ['y', 'xTyPPzTx', 'xTy']
#initial_op_list = ['y', 'xTyPPzTx', 'xPy']
#initial_op_list = ['xPy', 'y', 'xTyTTx']
#true_op = random.choice(initial_op_list)
#true_params = [np.random.rand() for i in range(n_pars)]

initial_op_list = ['x', 'y']


num_ops = len(initial_op_list)

for i in range(1):
#    true_op=initial_op_list[i%2]
#    true_op= initial_op_list[-1]
    true_op = 'xTxTTx'
    true_params = [np.random.rand()]
    qmd = QMD(
        initial_op_list=initial_op_list, 
        true_operator=true_op, 
        true_param_list=None, 
        num_particles=num_particles,
        num_experiments = num_experiments, 
        num_times_for_bayes_updates = num_times_bayes,
        qle=qle,
        num_probes=5,
        max_num_branches = 0,
        max_num_qubits = 10, 
        parallel = True,
        growth_generator='simple_ising'
    )
   # qmd.learnModelNameList(model_name_list=['z'], blocking=True, use_rq=False)
    qmd.runRemoteQMD(num_spawns=3)

    

No parameters passed, randomising

Running  QLE  for true operator  xTxTTx  with parameters :  [0.22375065357254942]
Model  x  not previously considered -- adding.
Initialization Ready
Model  y  not previously considered -- adding.
Initialization Ready
Model  x being learned
QHL for x
Initialization Ready
Initial time selected > 2.17410080969
Final time selected > 0.678342757802
Final Parameters mean and stdev:[ 1.00266977  0.56905372]
branch 0.0 Current =  0
Model  y being learned
QHL for y
Initialization Ready
Initial time selected > 0.851286682041
Final time selected > 0.730352316535
Final Parameters mean and stdev:[ 0.04894754  0.52905079]
branch 0.0 Current =  1
All models on branch 0 have finished learning.
Active branch bayes db set to 0 for branch 0
Bayes factor bw x / y = 1e+00
Neither model convincingly stronger than the other.
active_branches_bayes: branch 0 Current =  0
Reset active_branch_bayes for branch 0 as 1
Returned from remote bayes function
Checking branchID 0
Numbe

Checking branchID 0
Number compared in DB:  b'1'
branch ID has all models compared
Champion of branch  0  is  x
Generating new models according to best of last round:  ['x', 'y'] ; and options: ['x', 'y', 'z']
New models to add to new branch :  ['xTx', 'xTy', 'xTz', 'yTx', 'yTy', 'yTz']
Model xTx  previously considered.
Model xTy  previously considered.
Model xTz  previously considered.
Model yTx  previously considered.
Model yTy  previously considered.
Model yTz  previously considered.
Model  xTx being learned
QHL for xTx
Initialization Ready
Initial time selected > 1.77236775206
Final time selected > 4.1978673669
Final Parameters mean and stdev:[ 0.33571808  0.64204525]
branch 1.0 Current =  0
Model  xTy being learned
QHL for xTy
Initialization Ready
Initial time selected > 0.543874371385
Final time selected > 0.493309276055
Final Parameters mean and stdev:[ 0.4046791  0.6364632]
branch 1.0 Current =  1
Model  xTz being learned
QHL for xTz
Initialization Ready
Initial time selected >

NameError: name 'branch_id' is not defined

In [ ]:
qmd.NumModelPairsPerBranch

In [ ]:
for k in active_branches_learning_models.keys():
    print("k=",k)
    print(active_branches_learning_models.get(k))

In [ ]:
int(0.0)

In [ ]:
qmd.NumModelsPerBranch

In [ ]:
for branchID_bytes in active_branches_bayes.keys():

    branchID = int(branchID_bytes)
    print(branchID_bytes)
    print(branchID)
    print(active_branches_bayes.get(branchID))
    
    
    if int(active_branches_bayes.get(branchID)) == 1:
        print("fff")
    else:
        print("ggg")

In [ ]:

active_branches_bayes.keys()

In [ ]:
import math
math.factorial(4)

In [ ]:
active_branches_learning_models.get(0)

In [ ]:
int(active_branches_bayes.get(int(0)))